### Make a surface and domain file given a lat  lon 

In [13]:
from scipy.stats import qmc
import numpy as np
import xarray as xr
import csv
import pandas as pd
import os
import netCDF4 as nc4
import sys
from tempfile import TemporaryFile                                                                                                                                 
import argparse                                                                                                                                                     
import shutil                                                                                                                                                       
import tempfile 
import random


### Domain file

In [14]:
domain_full = '/global/cfs/cdirs/e3sm/inputdata/share/domains/domain.lnd.fv1.9x2.5_gx1v6.090206.nc'
domain_full = xr.open_dataset(domain_full,  decode_times=False)

lon_mids = domain_full.xc[0,0:144]
lat_mids = domain_full.yc[0:96,0]

print(lon_mids)

<xarray.DataArray 'xc' (ni: 144)>
array([  0. ,   2.5,   5. ,   7.5,  10. ,  12.5,  15. ,  17.5,  20. ,  22.5,
        25. ,  27.5,  30. ,  32.5,  35. ,  37.5,  40. ,  42.5,  45. ,  47.5,
        50. ,  52.5,  55. ,  57.5,  60. ,  62.5,  65. ,  67.5,  70. ,  72.5,
        75. ,  77.5,  80. ,  82.5,  85. ,  87.5,  90. ,  92.5,  95. ,  97.5,
       100. , 102.5, 105. , 107.5, 110. , 112.5, 115. , 117.5, 120. , 122.5,
       125. , 127.5, 130. , 132.5, 135. , 137.5, 140. , 142.5, 145. , 147.5,
       150. , 152.5, 155. , 157.5, 160. , 162.5, 165. , 167.5, 170. , 172.5,
       175. , 177.5, 180. , 182.5, 185. , 187.5, 190. , 192.5, 195. , 197.5,
       200. , 202.5, 205. , 207.5, 210. , 212.5, 215. , 217.5, 220. , 222.5,
       225. , 227.5, 230. , 232.5, 235. , 237.5, 240. , 242.5, 245. , 247.5,
       250. , 252.5, 255. , 257.5, 260. , 262.5, 265. , 267.5, 270. , 272.5,
       275. , 277.5, 280. , 282.5, 285. , 287.5, 290. , 292.5, 295. , 297.5,
       300. , 302.5, 305. , 307.5, 310. , 

In [15]:
lat = 9.154300000000
lon = -79.846100000000

if lon < 0 : 
    lon = 360 + lon
    print(lon)

abslat = np.abs(lat_mids - lat)
abslon = np.abs(lon_mids - lon)
c = np.maximum(abslon, abslat)


([xloc],[yloc]) = np.where(c == np.min(c))

lat_new = lat_mids[yloc]
print(lat_new.values)
lon_new = lon_mids[xloc]
print(lon_new.values)

280.1539
8.5263157894736
279.99999999999994


In [16]:
domain = domain_full.sel(ni=xloc,nj=yloc)
domain.to_netcdf('/pscratch/sd/j/jneedham/fates-tutorial/domain_bci_fates_tutorial.nc')


### Surface data file

In [17]:
surfdata_full = '/global/cfs/cdirs/e3sm/inputdata/lnd/clm2/surfdata_map/surfdata_1.9x2.5_simyr2000_c180306.nc'
surfdata_full = xr.open_dataset(surfdata_full,  decode_times=False)


In [18]:
surfdata_full.dims

Frozen({'lsmlat': 96, 'lsmlon': 144, 'nlevsoi': 10, 'natpft': 17, 'time': 12, 'lsmpft': 17, 'numurbl': 3, 'numrad': 2, 'nlevurb': 5})

In [19]:
surfdata = surfdata_full.sel(lsmlon=xloc,lsmlat=yloc)
surfdata.to_netcdf('/pscratch/sd/j/jneedham/fates-tutorial/surfdata_bci_fates_tutorial.nc')
